# Time Series Prediction with BQML and AutoML

**Objectives**
 1. Learn how to use BQML to create a classification time-series model using `CREATE MODEL`.
 2. Learn how to use BQML to create a linear regression time-series model.
 3. Learn how to use AutoML Tables to build a time series model from data in BigQuery.

## Set up environment variables and load necessary libraries

In [1]:
PROJECT = "your-gcp-project-here" # REPLACE WITH YOUR PROJECT NAME
BUCKET = "your-gcp-bucket-here" # REPLACE WITH YOUR BUCKET
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [2]:
%env
PROJECT = PROJECT
REGION = REGION

In [4]:
!python3 -m pip freeze | grep google-cloud-bigquery==1.6.1 || \
python3 -m pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Review the dataset

In the previous lab we created the data we will use modeling and saved them as tables in BigQuery. Let's examine that table again to see that everything is as we expect. Then, we will build a model using BigQuery ML using this table.

In [14]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  stock_market.eps_percent_change_sp500
LIMIT
  10

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction,consensus_EPS,reported_EPS,surprise
0,A,2008-07-01,35.04,35.57,35.36,-0.21,0.999157,0.999157,0.819511,1.038516,...,2166,-0.005904,-0.018204,0.012300,Agilent Technologies Inc,Health Care,UP,0.49,0.43,-12.24
1,A,2005-11-14,32.80,32.90,34.50,1.60,1.000000,0.929179,0.620061,1.004255,...,1506,0.048632,-0.003850,0.052482,Agilent Technologies Inc,Health Care,UP,0.37,0.38,2.70
2,A,2009-11-13,27.97,28.61,29.37,0.76,0.914715,0.864733,0.434813,0.959804,...,2513,0.026564,0.014468,0.012097,Agilent Technologies Inc,Health Care,UP,0.23,0.32,39.13
3,A,2003-05-19,16.10,15.51,16.04,0.53,1.047066,0.985171,0.699549,1.077369,...,877,0.034172,-0.001129,0.035301,Agilent Technologies Inc,Health Care,UP,-0.15,-0.15,0.00
4,A,2008-07-01,35.04,35.57,35.36,-0.21,0.999157,0.999157,0.819511,1.038516,...,2166,-0.005904,-0.018204,0.012300,Agilent Technologies Inc,Health Care,UP,0.41,0.36,-12.20
5,A,2008-07-01,35.04,35.57,35.36,-0.21,0.999157,0.999157,0.819511,1.038516,...,2166,-0.005904,-0.018204,0.012300,Agilent Technologies Inc,Health Care,UP,0.52,0.46,-11.54
6,A,2008-07-01,35.04,35.57,35.36,-0.21,0.999157,0.999157,0.819511,1.038516,...,2166,-0.005904,-0.018204,0.012300,Agilent Technologies Inc,Health Care,UP,0.39,0.32,-17.95
7,A,2003-08-18,21.78,22.46,24.29,1.83,0.935886,0.913179,0.483081,0.969724,...,940,0.081478,0.002611,0.078868,Agilent Technologies Inc,Health Care,UP,-0.07,-0.02,71.43
8,A,2005-05-16,21.85,21.56,22.48,0.92,1.013915,0.951299,0.912801,1.029685,...,1379,0.042672,0.006957,0.035714,Agilent Technologies Inc,Health Care,UP,0.21,0.20,-4.76
9,A,2012-05-14,38.87,38.90,40.51,1.61,1.014139,1.014139,0.755784,1.050386,...,3141,0.041388,-0.005746,0.047134,Agilent Technologies Inc,Health Care,UP,0.73,0.78,6.85


## Using BQML

### Create classification model for `direction`

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see another example.

**The query will take about two minutes to complete**


We'll start with creating a classification model to predict the `direction` of each stock. 

We'll take a random split using the `symbol` value. With about 500 different values, using `MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1` will give 30 distinct `symbol` values which corresponds to about 171,000 training examples. After taking 70% for training, we will be building a model on about 110,000 training examples.

In [7]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.direction_model OPTIONS(model_type = "logistic_reg",
    input_label_cols = ["direction"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  direction
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) <= 15 * 70

""


## Get training statistics and examine training info

After creating our model, we can evaluate the performance using the [`ML.EVALUATE` function](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality#step_four_evaluate_your_model). With this command, we can find the precision, recall, accuracy F1-score and AUC of our classification model.

In [8]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.direction_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      direction
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      tomorrow_close IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) > 15 * 70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) <= 15 * 85))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.439675,0.44307,0.472393,0.432496,1.083131,0.608674


We can also examine the training statistics collected by Big Query. To view training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [9]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.direction_model`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.318181,0.352341,0.2,4337
1,0,1,0.248417,0.336518,0.4,4509
2,0,2,0.169067,0.326026,0.8,4821


### Compare to simple benchmark

Another way to asses the performance of our model is to compare with a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicted the majority class. For the training dataset, the majority class is 'STAY'. The following query we can see how this naive strategy would perform on the eval set.

In [10]:
%%bigquery --project $PROJECT
#standardSQL
WITH
  eval_data AS (
  SELECT
    symbol,
    Date,
    Open,
    Close,
    close_MIN_prior_5_days,
    close_MIN_prior_20_days,
    close_MIN_prior_260_days,
    close_MAX_prior_5_days,
    close_MAX_prior_20_days,
    close_MAX_prior_260_days,
    close_AVG_prior_5_days,
    close_AVG_prior_20_days,
    close_AVG_prior_260_days,
    close_STDDEV_prior_5_days,
    close_STDDEV_prior_20_days,
    close_STDDEV_prior_260_days,
    direction
  FROM
    `stock_market.eps_percent_change_sp500`
  WHERE
    tomorrow_close IS NOT NULL
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) > 15 * 70
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) <= 15 * 85)
SELECT
  direction,
  (COUNT(direction)* 100 / (
    SELECT
      COUNT(*)
    FROM
      eval_data)) AS percentage
FROM
  eval_data
GROUP BY
  direction

,direction,percentage
0,UP,28.834356
1,DOWN,27.607362
2,STAY,43.558282


So, the naive strategy of just guessing the majority class would have accuracy of 0.5509 on the eval dataset, just below our BQML model. 

### Create regression model for `normalized change`

We can also use BigQuery to train a regression model to predict the normalized change for each stock. To do this in BigQuery we need only change the OPTIONS when calling `CREATE OR REPLACE MODEL`. This will give us a more precise prediction rather than just predicting if the stock will go up, down, or stay the same. Thus, we can treat this problem as either a regression problem or a classification problem, depending on the business needs.


In [11]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.price_model OPTIONS(model_type = "linear_reg",
    input_label_cols = ["normalized_change"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  normalized_change
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  normalized_change IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) <= 15 * 70

""


Just as before we can examine the evaluation metrics for our regression model and examine the training statistics in Big Query

In [12]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.price_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      normalized_change
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      normalized_change IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) > 15 * 70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15 * 100) <= 15 * 85))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.019295,0.000979,0.001047,0.011039,-0.230556,-0.218738


In [13]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.price_model`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.002133,0.005099,0.2,3429
1,0,1,0.001020,0.005017,0.4,4851


## Train a Time Series model using AutoML Tables

### Step 1. Launch AutoML

Within the GCP console, navigate to Tables in the console menu.

<img src='../assets/console_menu_tables.png' width='50%'>

### Step 2. Create a Dataset

Select New Dataset and give it a name like `stock_market`. In the section on Importing data, select the option to import your data from a BigQuery Table. Fill in the details for your project, the dataset ID, and the table ID.

<img src='../assets/import_data_options.png' width='50%'>

### Step 3. Import the Data

Once you have created the dataset you can then import the data. This will take a few minutes.

<img src='../assets/importing_data.png' width='50%'>

### Step 4. Train the model.

Once the data has been imported into the dataset. You can examine the Schema of your data, Analyze the properties and values of the features and ultimately Train the model. Here you can also determine the label column and features for training the model. Since we are doing a classifcation model, we'll use `direction` as our target column.

<img src='../assets/schema_analyze_train.png' width='80%'>

Under the `Train` tab you can choose the features to use when training. Select the same features as we used above. 

<img src='../assets/train_model.png' width='50%'>

### Step 5. Evaluate your model.

Training can take many hours. But once training is complete you can inspect the evaluation metrics of your model. Since this is a classification task, we can also adjust the threshold and explore how different thresholds will affect your evaluation metrics. Also on that page, we can explore the feature importance of the various features used in the model and view confusion matrix for our model predictions.

<img src='../assets/eval_metrics.png' width='80%'>

### Step 6. Predict with the trained model. 

Once the model is done training, navigate to the Models page and Deploy the model, so we can test prediction. 

<img src='../assets/deploy_model.png' width='80%'>

When calling predictions, you can call batch prediction jobs by specifying a BigQuery table or csv file. Or you can do online prediction for a single instance.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License